# Rappels de biologie

In [7]:

import numpy as np
import pickle as pkl
#import sklearn as skl
import hmmlearn as skl
from markov_tools import *


In [8]:
### Quelques constantes
DNA = ["A", "C", "G", "T"]
stop_codons = ["TAA", "TAG", "TGA"] 

In [9]:

def load_data():

    with open('genome_genes.pkl', 'rb') as f:
        data = pkl.load(f,encoding='latin1') 

    Xgenes  = data.get("genes") #Les genes, une array de arrays

    Genome = data.get("genome") #le premier million de bp de Coli

    Annotation = data.get("annotation") ##l'annotation sur le genome
    ##0 = non codant, 1 = gene sur le brin positif
    
    return data,Xgenes,Genome,Annotation

In [11]:

data,xgenes,genome,annotation =load_data()


In [12]:
len(genome)

1000000

In [13]:
xgenes[0][:6]
##6 premieres bp: ATGCGA

[0, 3, 2, 1, 2, 0]

In [14]:
xgenes[0][-6:]
#6 dernieres bp: GTCTAA

[2, 3, 1, 3, 0, 0]

In [15]:
genome[0:20]

array([0, 2, 1, 3, 3, 3, 3, 1, 0, 3, 3, 1, 3, 2, 0, 1, 3, 2, 1, 0])

Sous-question : 

Estimez les probabilités a et b pour la matrice de transition. On utilise pour cela le fait que le temps de séjour dans un état suit une loi géométrique. 
Pour les régions intergénique, on fait l'hypothèse que la longueur moyenne est de 200 bp.
Déduisez en la probabilité a

In [16]:
a = 0.995

Sous-question : 

Estimez la longueur moyenne des séquences des gène donnés dans Xgenes et déduisez en la probabilité 
b

In [17]:
lgm =  sum(len(xgenes[i]) for i in range(len(xgenes)))/ len(xgenes)
print(lgm)
b = 1 - 1/ lgm
print(1/lgm)
print(b)

941.544
0.0010620852557076462
0.9989379147442924


Sous-question : 

Estimez la distribution des quatre lettres sur tout le génome, pour la loi des observations de l'intergénique. Attention, on doit renvoyer une matrice de shape (1,4) .

In [18]:
def distribution_lettres(genome):
    dst = np.zeros((1,4))
    for i in genome:
        dst[0] [int(i)] +=1
        
    return dst/len(genome) 

#teste
dstl = distribution_lettres(genome)
print(dstl)
Binter = dstl

[[ 0.242654  0.248799  0.265139  0.243408]]


Sous-question : 

Estimez la distribution pour chacune des positions des codons et ranger le résultat dans une matrice à 3 lignes et 4 colonnes. Pour chaque gène on ignorerera le premier et le dernier codon. 

In [19]:

def distribution_position_codon(xgenes):
    """
    xgenes : est une liste de genes, chaque gene commence par un codon start a savoir ... et se termine par un codon stop a savoir 
    On veut connaitre la position de chaque base dans un tous les blocs de la sequence hormis la codon start et stop 
    
    """
    
    dst = np.zeros((3,4))

    bloc=0
    for j in range(len(xgenes)):

        for i in range(3,len(xgenes[j])-3,3):

            x = xgenes[j] [i:i+3]
            for index,val in enumerate(x):
                #print(index,val)
                dst[index][val] +=1
            bloc +=1


    return dst/bloc

In [21]:
Bgene= distribution_position_codon(xgenes)
Bgene


array([[ 0.25120572,  0.24063005,  0.35462469,  0.15353954],
       [ 0.29302417,  0.23050332,  0.18223301,  0.2942395 ],
       [ 0.18018394,  0.26191285,  0.29654511,  0.2613581 ]])

Sous-question : 
On peut maintenant initialiser la matrice de transition pour notre modèle et l'utiliser pour prédire les positions des gènes. 

In [22]:
B_m1 = np.vstack((Binter, Bgene))

model1._set_emissionprob(B_m1)

vsbce, pred = model1.decode(Genome) 
#vsbce contient la log vsbce
#pred contient la sequence des etats predits (valeurs entieres entre 0 et 3)

#on peut regarder la proportion de positions bien predites
#en passant les etats codant a 1
sp = pred
sp[np.where(sp>=1)] = 1
percpred1 = float(np.sum(sp == Annotation) )/ len(Annotation)

#percpred1
#Out[10]:  0.636212

NameError: name 'model1' is not defined

In [24]:

# intergene , position 0 ( codon start ) , position 1 ( acide aminé ), position 2 ( codon stop )

#distribution initiale, on commence dans l'intergenique
pi = np.array([1, 0, 0, 0])

# Matrice de transition
                # i , p0 ,p1,p2
a= np.array([   [1-a, a  , 0, 0], #i
                [0  , 0  , 1, 0], #p0
                [0  , 0  , 0, 1], #p1
                [b  , 1-b, 0, 0]  #p2
            ])
# Matrice d'observation
b = np.vstack((Binter, Bgene))

# pour utiliser le modèle plus loin:
s, logp = viterbi(genome,pi,a,b)

AttributeError: 'numpy.ndarray' object has no attribute 'log'